In [1]:
# -*- coding: utf-8 -*-
"""
교수 '리스트' 페이지 URL들을 미리 알고 있을 때의 간단 수집기 (requests + bs4)
출력: pcu_professors_bs.csv
"""
import re
import csv
import time
import pandas as pd
import openpyxl
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

In [2]:
file_path = 'prof_pages.xlsx'

# 엑셀 파일 읽기
df = pd.read_excel(file_path)
df = df [['department', 'link_address']]

LIST_URLS = df['link_address']  # 학과 교수소개 홈페이지

In [3]:
HEADERS = {"User-Agent": "Mozilla/5.0"}
OUTPUT = "pcu_professors_bs.csv"

In [4]:
def get(url):
    for _ in range(3):
        r = requests.get(url, headers=HEADERS, timeout=20)
        if r.status_code == 200:
            return r.text
        time.sleep(1)
    r.raise_for_status()

In [5]:
def collect_detail_urls(url): #학과 교수 상세보기 리스트 추출
    html = get(url)
    soup = BeautifulSoup(html, "html.parser")
    urls = set()

    for a in soup.select("a[href]"):
        href = a["href"]
        if re.search(r"/professor/\d+/\d+", href):
            urls.add(urljoin(url, href))
    return list(urls), soup

In [6]:
def get_df_dept(detail_urls, rows):
    for durl in detail_urls:
        # print(durl)
        html = get(durl)
        soup = BeautifulSoup(html, "html.parser")    
        dept = soup.select_one('meta[name="keywords"]')['content'].split(" ")[-1] #학과명
        name = soup.select_one("span.pho").select_one("img")["alt"].split()[-2] #교수명
        title = soup.select_one("span.pho").select_one("img")["alt"].split()[-1] #직위
        # print(name, title)
        print(dept)

        # 정규식 패턴: "04_off  111" 또는 "04_on  111"
        pattern = re.compile(r"04_(off|on)\s+\d+")
        edu_list = soup.find_all("li", class_=pattern)
        edu = [edu_li.text.strip() for edu_li in edu_list]

        # if soup.select_one("#txt > div > h4:nth-child(2)"): 
        #     ul_tag_edu = soup.select_one("#txt > div:nth-child(2) > ul:nth-child(3)")   
        # elif soup.select_one("#txt > div > h4"): 
        #     ul_tag_edu = soup.select_one("#txt > div > ul")
        # else:
        #     # raise Exception(f"{dept}의 {name} 교수 내용에서 에러 발생")
        #     print(f"{dept}의 {name} 교수 학력 내용에서 에러 발생")
        #     continue
        # # li 항목들을 리스트로 추출
        # edu = [li.text.strip() for li in ul_tag_edu.select("li")]
        # print(edu)

        # print(edu)
        # print("경력 및 활동")
        # 정규식 패턴: "05_off  111" 또는 "05_on  111"
        pattern = re.compile(r"05_(off|on)\s+\d+")
        career_list = soup.find_all("li", class_=pattern)
        career = [career_li.text.strip() for career_li in career_list]

        
        # if soup.select_one("#txt > div > h4:nth-child(4)"): 
        #     ul_tag_career = soup.select_one("#txt > div:nth-child(2) > ul:nth-child(5)")     
        # else:
        #     # raise Exception(f"{dept}의 {name} 교수 내용에서 에러 발생")
        #     print(f"{dept}의 {name} 교수 경력 내용에서 에러 발생")
        #     continue

        # # li 항목들을 리스트로 추출
        # career = [li.text.strip() for li in ul_tag_career.select("li")]
        # print(career)

        # print(career)
        # print()
        
        rows.append({
            "department": dept,
            "professor_name": name,
            "title": title,
            "education": edu,
            "career": career,
        })

    return rows

In [7]:
def get_df(url, rows):
    detail_urls, soup_list = collect_detail_urls(url)
    rows = get_df_dept(detail_urls, rows)
    return rows

In [20]:
rows = []
for list_url in LIST_URLS:
    print(list_url)
    rows = get_df(list_url, rows)
    
df = pd.DataFrame(rows)

https://hakgwa.pcu.ac.kr/ssm/250/professor/23300600
자율전공학부
https://hakgwa.pcu.ac.kr/lifelong/194/professor/23300000
평생교육융합학부
평생교육융합학부
평생교육융합학부
평생교육융합학부
평생교육융합학부
평생교육융합학부
평생교육융합학부
평생교육융합학부
https://hakgwa.pcu.ac.kr/gis/206/professor/23300300
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
글로벌자율융합학부
https://hakgwa.pcu.ac.kr/jlac/115/professor/23000200
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
https://hakgwa.pcu.ac.kr/jlac/154/professor/23001100
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시경교양대학
주시

In [23]:
df.tail(10)

,department,professor_name,title,education,career
91,주시경교양대학,유송이,교수,"[한국교원대학교 수료(해당학위없음)박사수료(미술교육전공 전공), 충남대학교 미술학석...","[현) 배재대학교 주시경교양대학 중점교육부 조교수, 충남대학교 예술대학 회화과 강사]"
92,주시경교양대학,구상욱,교수,"[서울대학교 경영학박사(국제경영전략 전공), 미국 Univ. of Michigan ...","[현) 배재대학교 벤처창업학 전공주임교수, 피츠버그대학교(UNIVERSITY OF ..."
93,주시경교양대학,문정현,교수,[배재대학교 문학박사(한국어교육학 전공)],"[(현재) 배재대학교 주시경교양대학 기초교육부교수, (현재) 배재대학교 주시경교양교..."
94,주시경교양대학,박현이,교수,[충남대학교 문학박사(현대문학 전공)],"[현) 배재대학교 주시경교양대학 미래역량교육부 교수, 충남대학교 초빙교수, 목원대학..."
95,주시경교양대학,조선문,교수,[인하대학교 공학박사(컴퓨터정보공학 전공)],[현) 배재대학교 주시경교양대학 교수]
96,주시경교양대학,조세린,교수,"[미국 Harvard Univ. 사회학박사(동아시아 언어 및 문명 전공), 미국 H...",[2017~ Official Apprentice (Jonsuja 전수자 (傳受者))...
97,주시경교양대학,이수현,교수,"[연세대학교 문학박사(한국어교육정보학 전공), 연세대학교 문학박사(한국어교육정보학 ...","[(현재)배재대학교 주시경교양대학 기초교육부 교수, 배재대학교 국어국문한국어교육학과..."
98,주시경교양대학,박옥희,교수,"[미국 뉴욕주립대학교 이학석사(교육과정개발 및 교육공학 전공), 한국교원대학교 교육...",[현) 배재대학교 주시경교양대학 기초교육부 교수]
99,주시경교양대학,김성례,교수,[한국외국어대학교 국제관계학박사(사회·문화전공)],"[현)배재대학교 주시경교양대학 기초교육부 교수, 조선대학교 초빙객원교수/호남신학대학..."
100,주시경교양대학,리마니치,교수,"[배재대학교 교육학석사(TESOL 교육 전공), 캐나다 Univ. of Windso...",[]


In [24]:
df.to_excel("output.xlsx", index=False)

In [27]:
# 여기까지